# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [6]:
import pandas as pd
import numpy as np

---
## 2. Load data

In [7]:
df = pd.read_csv('clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [8]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [9]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [10]:
# Calculate means of the new variables grouped by whether the companies churned or not
merged = pd.merge(df, diff, on = 'id')
sens = merged.groupby('churn').agg({'offpeak_diff_dec_january_energy': 'mean', 'offpeak_diff_dec_january_power' : 'mean'})
sens

,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
churn,,
0,-0.004562,0.277320
1,-0.004605,0.289483


Even with this feature there still seems to be little predicive power for price elasticity based on the mean

## Creating Final DataFrame

In [11]:
#Use the changes within the periods to get a better understanding of the price elasticity effect
period_diff = price_df.groupby('id').agg({'price_off_peak_var' : 'mean', 'price_peak_var' : 'mean', 'price_mid_peak_var' : 'mean', 
                            'price_off_peak_fix' : 'mean', 'price_peak_fix' : 'mean', 'price_mid_peak_fix' : 'mean'})
period_diff['off_peak_peak_diff_energy'] = period_diff['price_off_peak_var'] - period_diff['price_peak_var']
period_diff['peak_mid_diff_energy'] = period_diff['price_peak_var'] - period_diff['price_mid_peak_var']
period_diff['off_peak_mid_diff_energy'] = period_diff['price_off_peak_var'] - period_diff['price_mid_peak_var']

period_diff['off_peak_peak_diff_power'] = period_diff['price_off_peak_fix'] - period_diff['price_peak_fix']
period_diff['peak_mid_diff_power'] = period_diff['price_peak_fix'] - period_diff['price_mid_peak_fix']
period_diff['off_peak_mid_diff_power'] = period_diff['price_off_peak_fix'] - period_diff['price_mid_peak_fix']

period_diff


,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix,off_peak_peak_diff_energy,peak_mid_diff_energy,off_peak_mid_diff_energy,off_peak_peak_diff_power,peak_mid_diff_power,off_peak_mid_diff_power
id,,,,,,,,,,,,
0002203ffbb812588b632b9e628cc38d,0.124338,0.103794,0.073160,40.701732,24.421038,16.280694,0.020545,0.030633,0.051178,16.280694,8.140345,24.421038
0004351ebdd665e6ee664792efc4fd13,0.146426,0.000000,0.000000,44.385450,0.000000,0.000000,0.146426,0.000000,0.146426,44.385450,0.000000,44.385450
0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.000000,0.000000,45.319710,0.000000,0.000000,0.181558,0.000000,0.181558,45.319710,0.000000,45.319710
0010ee3855fdea87602a5b7aba8e42de,0.118757,0.098292,0.069032,40.647427,24.388455,16.258971,0.020465,0.029260,0.049725,16.258972,8.129484,24.388456
00114d74e963e47177db89bc70108537,0.147926,0.000000,0.000000,44.266930,0.000000,0.000000,0.147926,0.000000,0.147926,44.266930,0.000000,44.266930
...,...,...,...,...,...,...,...,...,...,...,...,...
ffef185810e44254c3a4c6395e6b4d8a,0.138863,0.115125,0.080780,40.896427,24.637456,16.507972,0.023739,0.034344,0.058083,16.258971,8.129484,24.388455
fffac626da707b1b5ab11e8431a4d0a2,0.147137,0.000000,0.000000,44.311375,0.000000,0.000000,0.147137,0.000000,0.147137,44.311375,0.000000,44.311375
fffc0cacd305dd51f316424bbb08d1bd,0.153879,0.129497,0.094842,41.160171,24.895768,16.763569,0.024382,0.034655,0.059037,16.264402,8.132199,24.396601


In [12]:
period_diff = period_diff.drop(columns = ['price_off_peak_var', 'price_peak_var', 'price_mid_peak_var', 'price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix'])
period_diff

,off_peak_peak_diff_energy,peak_mid_diff_energy,off_peak_mid_diff_energy,off_peak_peak_diff_power,peak_mid_diff_power,off_peak_mid_diff_power
id,,,,,,
0002203ffbb812588b632b9e628cc38d,0.020545,0.030633,0.051178,16.280694,8.140345,24.421038
0004351ebdd665e6ee664792efc4fd13,0.146426,0.000000,0.146426,44.385450,0.000000,44.385450
0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.000000,0.181558,45.319710,0.000000,45.319710
0010ee3855fdea87602a5b7aba8e42de,0.020465,0.029260,0.049725,16.258972,8.129484,24.388456
00114d74e963e47177db89bc70108537,0.147926,0.000000,0.147926,44.266930,0.000000,44.266930
...,...,...,...,...,...,...
ffef185810e44254c3a4c6395e6b4d8a,0.023739,0.034344,0.058083,16.258971,8.129484,24.388455
fffac626da707b1b5ab11e8431a4d0a2,0.147137,0.000000,0.147137,44.311375,0.000000,44.311375
fffc0cacd305dd51f316424bbb08d1bd,0.024382,0.034655,0.059037,16.264402,8.132199,24.396601


In [13]:
#convert orgin_up variable to a binary integer
df = df.replace(['f', 't'], [0, 1])

In [14]:
#Aggregate count ids by different origins
df.groupby(['origin_up']).agg({'id' : 'count', 'churn':'mean'})

,id,churn
origin_up,,
MISSING,64,0.062500
ewxeelcelemmiwuafmddpobolfuxioce,1,0.000000
kamkkxfxxuwbdslkwifmmcsiusiuosws,4294,0.060084
ldkssxwpmemidmecebumciepifcamkci,3148,0.083863
lxidpiddsbxsbosboudacockeimpuepw,7096,0.125846
usapbepcfoloekilkwsdiboslwaxobdp,2,0.000000


In [15]:
#low usage of 2 of the origins so we can drop those indexes
drops = df[(df['origin_up'] == 'ewxeelcelemmiwuafmddpobolfuxioce') | (df['origin_up'] == 'usapbepcfoloekilkwsdiboslwaxobdp')]
df.drop(drops.index, inplace = True)
df

,Unnamed: 0,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,...,mean_3m_price_p1_var,mean_3m_price_p2_var,mean_3m_price_p3_var,mean_3m_price_p1_fix,mean_3m_price_p2_fix,mean_3m_price_p3_fix,mean_3m_price_p1,mean_3m_price_p2,mean_3m_price_p3,churn
0,0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,...,0.131756,0.092638,0.036909,42.497907,12.218665,8.145777,42.629663,12.311304,8.182687,1
1,1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,...,0.147600,0.000000,0.000000,44.444710,0.000000,0.000000,44.592310,0.000000,0.000000,0
2,2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,...,0.167798,0.088409,0.000000,44.444710,0.000000,0.000000,44.612508,0.088409,0.000000,0
3,3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,...,0.148586,0.000000,0.000000,44.444710,0.000000,0.000000,44.593296,0.000000,0.000000,0
4,4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,...,0.119906,0.101673,0.073719,40.728885,24.437330,16.291555,40.848791,24.539003,16.365274,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14600,14600,18463073fb097fc0ac5d3e040f356987,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,2012-05-24,2016-05-08,2015-05-08,2014-05-26,...,0.140172,0.000000,0.000000,44.444710,0.000000,0.000000,44.584882,0.000000,0.000000,0
14601,14601,d0a6f71671571ed83b2645d23af6de00,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,2012-08-27,2016-08-27,2012-08-27,2015-08-28,...,0.102352,0.093181,0.070990,59.173468,36.490689,8.367731,59.275820,36.583870,8.438721,1
14602,14602,10e6828ddd62cbcf687cb74928c4c2d2,foosdfpfkusacimwkcsosbicdxkicaua,1844,0,179,2012-02-08,2016-02-07,2012-02-08,2015-02-09,...,0.119906,0.101673,0.073719,40.728885,24.437330,16.291555,40.848791,24.539003,16.365274,1
14603,14603,1cf20fd6206d7678d5bcafd28c53b4db,foosdfpfkusacimwkcsosbicdxkicaua,131,0,0,2012-08-30,2016-08-30,2012-08-30,2015-08-31,...,0.147600,0.000000,0.000000,44.444710,0.000000,0.000000,44.592310,0.000000,0.000000,0


In [16]:
#dummy variable creation for new variables
df = pd.get_dummies(data = df, columns = ['origin_up'], dtype = int )
df = df.rename(columns = {'origin_up_MISSING' : 'Missing', 'origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws':'campaign1', 
                    'origin_up_ldkssxwpmemidmecebumciepifcamkci':'campaign2', 'origin_up_lxidpiddsbxsbosboudacockeimpuepw': 'campaign3'})


In [17]:
#merge to create final Data Frame to run classifier
final_for_model = pd.merge(df, period_diff, on = 'id')
final_for_model = final_for_model[['id', 'off_peak_peak_diff_energy', 'peak_mid_diff_energy', 'off_peak_mid_diff_energy',
                                  'off_peak_peak_diff_power', 'peak_mid_diff_power', 'off_peak_mid_diff_power', 
                                   'has_gas','Missing', 'campaign1', 'campaign2', 'campaign3', 'churn']]

#decided to drop "Missing" because only 64 customers from that campaign
final_for_model = final_for_model.drop(columns = ['Missing'])
final_for_model

,id,off_peak_peak_diff_energy,peak_mid_diff_energy,off_peak_mid_diff_energy,off_peak_peak_diff_power,peak_mid_diff_power,off_peak_mid_diff_power,has_gas,campaign1,campaign2,campaign3,churn
0,24011ae4ebbe3035111d65fa7c15bc57,0.024038,0.034219,0.058257,18.590255,7.450670,26.040925,1,0,0,1,1
1,d29c2c54acc38ff3c0614d0a653813dd,0.142485,0.007124,0.149609,44.311375,0.000000,44.311375,0,1,0,0,0
2,764c75f661154dac3a6c254cd082ea7d,0.082090,0.088421,0.170512,44.385450,0.000000,44.385450,0,1,0,0,0
3,bba03439a292a1e166f80264c16191cb,0.151210,0.000000,0.151210,44.400265,0.000000,44.400265,0,1,0,0,0
4,149d57cf92fc41cf94415803a877cb4b,0.020536,0.030773,0.051309,16.275263,8.137629,24.412893,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14597,18463073fb097fc0ac5d3e040f356987,0.144124,0.000000,0.144124,44.370635,0.000000,44.370635,1,0,0,1,0
14598,d0a6f71671571ed83b2645d23af6de00,0.011393,0.024589,0.035982,22.622294,28.047961,50.670256,0,0,0,1,1
14599,10e6828ddd62cbcf687cb74928c4c2d2,0.020545,0.030633,0.051178,16.280694,8.140345,24.421038,0,0,0,1,1
14600,1cf20fd6206d7678d5bcafd28c53b4db,0.142485,0.007124,0.149609,44.311375,0.000000,44.311375,0,0,0,1,0


# Random Forest Classifier

In [23]:
from sklearn.ensemble import RandomForestClassifier
X = final_for_model.drop(['id', 'churn'], axis = 1)
Y = final_for_model['churn']
#Split dataset into train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.7, random_state = 19, stratify = Y)
#Fit the classifier
churn_classifier = RandomForestClassifier(n_estimators = 1000, class_weight = 'balanced_subsample')
churn_classifier.fit(X_train, Y_train)
predicted_y = churn_classifier.predict(X_test)
#random_state = 19, max_depth = 5, max_features = None, class_weight = 'balanced_subsample'

In [24]:
#Import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
#Evaluate Model Performance
accuracy_score(Y_test, predicted_y)

0.7763067792741383

In [25]:
confusion_matrix(Y_test, predicted_y, normalize = 'true')


array([[0.83160556, 0.16839444],
       [0.7370892 , 0.2629108 ]])

In [26]:
confusion_matrix(Y_test, predicted_y, normalize = 'all')

array([[0.75074184, 0.15202009],
       [0.07167313, 0.02556494]])

In [27]:
print(classification_report(Y_test, predicted_y))

              precision    recall  f1-score   support

           0       0.91      0.83      0.87      3955
           1       0.14      0.26      0.19       426

    accuracy                           0.78      4381
   macro avg       0.53      0.55      0.53      4381
weighted avg       0.84      0.78      0.80      4381



# Takeaways

The model did not perform very well after I balanced the class weights in the classifier. At first, I had gotten an accuracy score of 90%, only to find that the model did not predict any churns due to the weights in the bootstrapped samples. The final model only predicted 56% of the true non-churns and a surprising 61.5% of true churns. In a data set with only 10% total churn rate, 61.5% prediction rate is suboptimal for predicting churn of future customers. 

I chose these evaluation metrics because they give a good understanding of how the model performed against a test dataset. The classification matrix as a percentage of correct predictions allows me to understand which areas need to improve for a future performance. 
The Random Forest Classifier is a good model choice in this case due to the binary output variable of churn. The decision trees come to a consensus conclusion based on the data input, and it comes from a bootstrapped sample to eliminate much of the variance

Overall, I don't think this model performance is satisfactory for the needs of the company. The prediction rate of 61.5% of churn is not terrible, but with the model getting more than 40% of its total predictions wrong, it leaves little confidence that it is trained properly with well-engineered variables. I will have to continue to work to improve the value this model can provide.